In [2]:
import os, json, requests, time, yaml
from pprint import pprint
from datetime import datetime

with open("../key.yaml", "r") as file:
    api_keys = yaml.safe_load(file)

api_key = api_keys["API_KEY"]
api_username = "ShubhamG"
defaultHost = "discourse.onlinedegree.iitm.ac.in"

# Headers for authentication
headers = {
    'Api-Key': api_key,
    'Api-Username': api_username
}
# Dictionary to store the data
slug_id_pairs = {"mlt-kb": 32, "maths2-kb": 23, "dlp-kb": 53}

# getting posts for MLT

In [4]:
num_posts_dict = { "subject": [], "total_topics": [] }
for slug, id in slug_id_pairs.items():
    num_topics=0
    page=0
    comparison_date_1 = datetime.strptime("2024-09-24", "%Y-%m-%d")
    comparison_date_2 = datetime.strptime("2024-12-29", "%Y-%m-%d")
    # endpoint = "https://discourse.onlinedegree.iitm.ac.in/c/courses/softengg-kb/56.json?page=0"
    topic_ids = list()
    created_time_list = list()
    for page in range(30):
        time.sleep(0.9)
        endpoint = f"https://{defaultHost}/c/courses/{slug}/{id}.json?page={page}"
        response = requests.get(endpoint, headers=headers)
        if response.status_code == 200:
            data = response.json()
            topics = data["topic_list"]["topics"]
            for topic in topics:
                created_at = datetime.strptime(topic["created_at"], "%Y-%m-%dT%H:%M:%S.%fZ")
                topic_id = topic["id"]
                """
                if topic_id not in topic_ids:
                    topic_ids.append(topic_id)
                else:
                    print(f"Duplicate topic ID found: {topic_id}")
                if created_at not in created_time_list:
                    created_time_list.append(created_at)
                else:
                    print(f"Duplicate created_at found: {created_at} with ID = {topic_id}")
                """
                if (created_at > comparison_date_1) and (created_at < comparison_date_2) and (topic_id not in topic_ids):
                    num_topics += 1


        else:
            print(f"Failed to retrieve data.\nStatus code: {response.status_code}\nText: {response.text}")

    num_posts_dict["subject"].append(slug.split("-")[0])
    num_posts_dict["total_topics"].append(num_topics)
    print(f"num_topics for {slug} = {num_topics}")

num_posts_dict

num_topics for mlt-kb = 191
num_topics for maths2-kb = 347
num_topics for dlp-kb = 107


{'subject': ['mlt', 'maths2', 'dlp'], 'total_topics': [191, 347, 107]}

In [ ]:
for i in range(0,len(created_time_list),30):
    print(i,created_time_list[i], sep="\t")

0	2025-01-10 07:15:54.375000
30	2024-12-30 12:03:19.698000
60	2024-12-16 17:09:22.535000
90	2024-12-10 20:46:03.806000
120	2024-11-24 08:47:24.778000
150	2024-11-01 17:33:36.538000
180	2024-10-17 10:55:19.872000
210	2024-09-28 14:16:46.259000
240	2024-09-05 15:51:49.359000
270	2022-11-25 18:39:20.637000
300	2024-08-05 13:52:46.019000
330	2024-08-02 06:48:56.083000
360	2024-07-25 04:09:43.056000
390	2024-07-06 22:28:22.958000
420	2024-07-02 17:21:20.802000
450	2024-06-13 02:35:33.459000
480	2024-04-27 16:32:44.021000
510	2024-04-23 22:22:02.480000
540	2024-04-11 04:58:02.630000
570	2024-03-22 10:36:05.482000
600	2024-03-18 02:28:28.478000
630	2024-03-07 05:50:57.512000
660	2024-02-23 10:43:01.133000
690	2024-02-13 19:15:02.980000
720	2024-02-05 06:05:46.745000
750	2024-01-21 09:16:45.568000
780	2023-12-26 11:13:41.313000
810	2022-06-02 18:59:55.539000
840	2023-12-11 06:47:14.438000
870	2023-12-02 14:38:06.632000
900	2022-10-31 16:00:30.678000
930	2023-11-10 08:39:26.245000
960	2023-10-3

In [6]:
# Dump total posts per subject in a csv file
import pandas as pd
df = pd.DataFrame(num_posts_dict)
df.to_csv('total_posts_per_subject.csv')